## Code along - read from excel

In [ ]:
import pandas as pd

df = pd.read("data/calories.csv")
df.head() 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df["FoodCategory"].unique() # för att se alla unika kategorier

In [ ]:
df["FoodCategory"].value_counts() # för att räkna hur många values det finns i kategorierna

In [ ]:
df["per100grams"].value_counts() 

## Data cleaning

In [ ]:
df.columns()

In [ ]:
# för att ädra nman så lägga in en dictionary med gamla namnet som en nyckel, och nya namnet som value
# axis =1 (axis=columns) används för att den kommer börja från kolumn 1, och ist för index 0
df_cleaned_names = df.rename(
    {"Cals_per100grams":"Calories", "KJ_per100grams": "KJ", "Per100grams": "per100"},
    axis=1,
)

df_cleaned_names.head()

In [ ]:
# använder .str för att arbeta med strängar, så vi kan ta bort strängarna
df_cleaned_names["calories"] = df_cleaned_names["calories"].str[:-3].astype(int)
df_cleaned_names.head()

In [ ]:
# för att få info om hur filen ser uut nu
df_cleaned_names.info()

In [ ]:

df_cleaned_names["per100"].value_counts()

In [ ]:
# splitta på liquids och solids
# gör en booleans mask - som innehåller massa True & False (kan använda den för att filtrera)
df_cleaned_names["per100"] == "100ml"

In [ ]:
# får ut alla som är True = alla liquids. 423 st.
df_liquids = df_cleaned_names[df_cleaned_names["per100"] == "100ml"]
df_liquids.head()

In [ ]:
# motsvarande- men strängar. lägger in column och vad den ska vara lika med (100g)
# fungerar bra vid större dataset.
df_solids = df_cleaned_names.query("per100 == '100g'")
df_solids.head()

## Fint top 5 categories of highest calories

In [ ]:
# .sort_values är motsvarande order by. kommer automatiskt i ascending order.
df_solids_sorted = df_solids.sort_values(by = "calories", ascending = False)
# .iloc[:5] får dom 5 första (alltså de största)
df_solids_top5 = df_solids_sorted.iloc[:5]
df_solids_top5.head()

In [ ]:
df_liquids_top5 = df_liquids.sort_values(by= "calories", ascending = False).head()
df_liquids_top5

In [ ]:
# gruppera 
df_top5_categories = df_cleaned_names.groupby("FoodCategory")["calories"].median().sort_values(ascending = False).reset_index().head()

df_top5_categories

In [7]:
query = """ SELECT
                FoodCategory,
                MEDIAN(calories) AS median_calories
            FROM 
                df_cleaned_names
            GROUP BY
                FoodCategory
            ORDER BY
                median_calories
            DESC
            LIMIT 
                5
            
"""

In [ ]:
import duckdb

duckdb.query(query)

In [ ]:
top5_category = duckdb.query(query).df()
top5_category